This Notebook is to build our the first phase of duckDB in the check new module 

In [2]:
import duckdb

In [69]:
# gw_con = duckdb.connect(database='database/grandwelcome.duckdb', read_only=False)
# gw_con.execute("Import DATABASE 'database'")

In [49]:
import duckdb

def checkNewRes(res_df):
    # create gwdb
    gw_con = duckdb.connect(database='database/grandwelcome.db', read_only=False) 
    gw_con.execute("CREATE TABLE online AS SELECT * FROM res_df")
    new=gw_con.execute("Select * from online where Res_ID NOT IN (Select Res_ID from gw_res)").df()   
    gw_con.execute("drop table IF EXISTS online")
    if len(new) > 0:
        new
        addnew = gw_con.df(new)
        # add new to database 
        addnew.insert_into("gw_res")   
        gw_con.execute("Export DATABASE 'database'")
    gw_con.close()
    return new
    

In [112]:
gw_con.execute("Show Tables").fetchall()

[('breakdown_dashboard',), ('cfrm',), ('gw_res',)]

In [105]:
gw_con.execute("Import DATABASE 'database'")

CatalogException: Catalog Error: Table with name "breakdown_dashboard" already exists!

In [106]:
gw_con.execute("Export DATABASE 'database' (FORMAT PARQUET)")

In [121]:
gw_con.close()

In [51]:
    # create a summary table
#     gw_con = duckdb.connect(database='database/grandwelcome.duckdb', read_only=False)
    summ1=gw_con.execute('Select Res_ID, "Check-In",Checkout, Income from gw_res').df()
    # summ1=summ[['Res_ID','Check-In','Checkout', 'Nights', 'Income']]
    

In [61]:
# summ1
import re
from dateutil.parser import parse
from datetime import timedelta
import pandas as pd

In [74]:
    nightly_rate = []
    for i,b in summ1.iterrows():
        Res_ID = b['Res_ID']
        Arrival = parse(b['Check-In']).date()
        Departure = parse(b['Checkout']).date()
        Income_raw = b['Income']
        Income= float(re.sub(r'[^\d.]', '', Income_raw))
        times = [Arrival + timedelta(days=x) for x in range((Departure-Arrival).days + 1)]
        for t in times:
            nightly = Income / len(times)
            night_form = '{0:.2f}'.format(nightly)
            t_stmp=pd.Timestamp(t)
            nightly_rate.append([Res_ID,t_stmp,float(night_form)])
    breakdown = pd.DataFrame(nightly_rate, columns = ['Res_ID','Date', 'Per Night'])
    breakdown['Month-Year']=breakdown['Date'].dt.strftime('%m/%Y')
        

In [ ]:
gw_con.execute("CREATE TABLE breakdown_dashboard AS SELECT * FROM breakdown")

In [97]:
addbreakdn = gw_con.df(breakdown)
# add new to database 
addbreakdn.insert_into("breakdown_dashboard")

In [99]:
gw_con.execute("Select * from breakdown_dashboard where Res_ID = 36667491").fetchall()

[(36667491, datetime.datetime(2022, 7, 21, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 22, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 23, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 24, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 25, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 26, 0, 0), 181.26, '07/2022'),
 (36667491, datetime.datetime(2022, 7, 27, 0, 0), 181.26, '07/2022')]

In [98]:
gw_con.execute("Select * from breakdown_dashboard").fetchall()

[(1096275, datetime.datetime(2022, 9, 23, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 24, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 25, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 26, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 27, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 28, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 29, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 9, 30, 0, 0), 0.0, '09/2022'),
 (1096275, datetime.datetime(2022, 10, 1, 0, 0), 0.0, '10/2022'),
 (1096275, datetime.datetime(2022, 10, 2, 0, 0), 0.0, '10/2022'),
 (1096930, datetime.datetime(2022, 10, 14, 0, 0), 93.71, '10/2022'),
 (1096930, datetime.datetime(2022, 10, 15, 0, 0), 93.71, '10/2022'),
 (1096930, datetime.datetime(2022, 10, 16, 0, 0), 93.71, '10/2022'),
 (1096930, datetime.datetime(2022, 10, 17, 0, 0), 93.71, '10/2022'),
 (1096930, datetime.datetime(2022, 10, 18, 0, 0), 93.71, '10/202

In [81]:
addbreakdn

---------------------
--- Relation Tree ---
---------------------
pandas_scan(0x7f4ab5d9f8b0)

---------------------
-- Result Columns  --
---------------------
- Res_ID (BIGINT)
- Date (TIMESTAMP)
- Per Night (DOUBLE)
- Month-Year (VARCHAR)

---------------------
-- Result Preview  --
---------------------
Res_ID	Date	Per Night	Month-Year	
BIGINT	TIMESTAMP	DOUBLE	VARCHAR	
[ Rows: 10]
1096275	2022-09-23 00:00:00	0.0	09/2022
1096275	2022-09-24 00:00:00	0.0	09/2022
1096275	2022-09-25 00:00:00	0.0	09/2022
1096275	2022-09-26 00:00:00	0.0	09/2022
1096275	2022-09-27 00:00:00	0.0	09/2022
1096275	2022-09-28 00:00:00	0.0	09/2022
1096275	2022-09-29 00:00:00	0.0	09/2022
1096275	2022-09-30 00:00:00	0.0	09/2022
1096275	2022-10-01 00:00:00	0.0	10/2022
1096275	2022-10-02 00:00:00	0.0	10/2022



In [93]:
gw_con.execute('CREATE OR REPLACE TABLE breakdown_dashboard(Res_ID BIGINT,Date TIMESTAMP,"Per Night" DOUBLE,"Month-Year" VARCHAR, PRIMARY KEY(Res_ID, Date,"Per Night", "Month-Year"))')

In [102]:
gw_con.execute('CREATE OR REPLACE TABLE  cfrm("Res_ID" BIGINT PRIMARY KEY, "Status" VARCHAR, "Unit" VARCHAR, "Guest" VARCHAR, "Booked Date" VARCHAR, "Check-In" VARCHAR, "Checkout" VARCHAR, "Nights" BIGINT, "Income" VARCHAR)')
gw_con.execute('CREATE OR REPLACE TABLE  gw_res("Res_ID" BIGINT PRIMARY KEY, "Status" VARCHAR, "Unit" VARCHAR, "Guest" VARCHAR, "Booked Date" VARCHAR, "Check-In" VARCHAR, "Checkout" VARCHAR, "Nights" BIGINT, "Income" VARCHAR)')

In [110]:
gw_con.execute("load.sql")

ParserException: Parser Error: syntax error at or near "."
LINE 1: load.sql
            ^

In [101]:
gw_con.execute('CREATE TABLE cfrm("Res_ID" BIGINT PRIMARY KEY, "Status" VARCHAR, "Unit" VARCHAR, "Guest" VARCHAR, "Booked Date" VARCHAR, "Check-In" VARCHAR, "Checkout" VARCHAR, "Nights" BIGINT, "Income" VARCHAR')

ParserException: Parser Error: syntax error at end of input

In [108]:
gw_con.execute("Select * from cfrm").fetchall()

[]

# Reload Data

In [119]:

gw_con.execute("COPY cfrm FROM 'database/cfrm.parquet' (FORMAT 'parquet')").fetchall()


[(0,)]

In [114]:
gw_con.execute("Show Tables").fetchall()
gw_con.execute("Select * from cfrm").fetchall()

[]

In [111]:
cfrm_df = pd.read_csv('database/cfrm.csv')
gw_res_df = pd.read_csv('database/gw_res.csv')
addgw = gw_con.df(gw_res_df)
addcf = gw_con.df(cfrm_df)

# add new to database 
addgw.insert_into("gw_res")
addcf.insert_into("cfrm")

EmptyDataError: No columns to parse from file

In [120]:
gw_con.execute("Select * from cfrm").fetchall()
# gw_con.execute("Select * from gw_res").fetchall()

[]